In [1]:
import os
import time
import datetime
import copy
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib import cm

# 关闭警告提醒
warnings.filterwarnings("ignore")
# 显示所有列
pd.set_option('display.max_columns', None)

# 设置英文字体为Times New Roman
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
plt.rcParams['font.size'] = 15  # 设置全局字体大小

# 设置中文字体为宋体
chinese_font = FontProperties(fname='C:\Windows\Fonts\simsun.ttc', size=15)  # 指定字体文件路径

In [7]:
# 读取速度估计预处理数据 2021年12月1日 佛山市中心城区燃油客车、货车路径重构数据 对应每一单次出行

folder_path = r'E:\硕士毕业论文update250120\案例分析\数据\路径重构结果\有效处理结果'
test_data = pd.read_csv(folder_path + '/' + 'travel_info.csv', encoding='gbk')
test_data['start_time'] = pd.to_datetime(test_data['start_time'])
test_data['stop_time'] = pd.to_datetime(test_data['stop_time'])

# 按照客车、货车分类
bus_test_data = test_data[test_data['veh_type'].str.contains('客车')]
truck_test_data = test_data[test_data['veh_type'].str.contains('货车')]
lighttruck_test_data = truck_test_data[truck_test_data['veh_type'].isin(['微型货车', '轻型货车'])]
heavytruck_test_data = truck_test_data[truck_test_data['veh_type'].isin(['中型货车', '重型货车'])]

print('2021年12月1日 总出行次数:', len(test_data))
print('客车占比: {:.2%}'.format(len(bus_test_data) / len(test_data)),
      '货车占比: {:.2%}'.format(len(truck_test_data) / len(test_data)))

bus_test_data.head(3)

2021年12月1日 总出行次数: 264422
客车占比: 97.74% 货车占比: 2.26%


,CLTMBH,start_time,stop_time,travel_time_s,predtravel_time_s,distance_km,speed_kmh,bayonet_records,node_records,start_bayonet,stop_bayonet,start_node_lon,start_node_lat,stop_node_lon,stop_node_lat,position,veh_type,fuel_type,pfbz,travel_error
0,粤E00003c875c35df53f1cc6bb188017915,2021-12-01 09:42:43,2021-12-01 10:03:39,1256.0,1644.0,17.77,50.94,4,229,440615000000324015,440604678881002001,113.149607,23.011112,113.063593,23.044394,2,小型客车,汽油,国5,总时间误差满足
1,粤E00003c875c35df53f1cc6bb188017915,2021-12-01 19:19:44,2021-12-01 19:36:09,985.0,1237.0,12.28,44.87,4,217,440604600031007003,440604305050003500,113.139246,23.015031,113.070463,23.047882,3,小型客车,汽油,国5,总时间误差满足
2,粤E000190caca7cf57954e46c53740d2a61,2021-12-01 21:22:15,2021-12-01 21:31:30,555.0,687.0,6.82,44.21,4,64,440604600031007003,440604679361002000,113.139246,23.015031,113.114211,23.004786,6,小型客车,汽油,国6,总时间误差满足


1. 24小时出行流量变化

In [9]:
def hour_trafficflow(df):
    # 提取小时
    df['start_hour'] = df['start_time'].dt.hour
    df['stop_hour'] = df['stop_time'].dt.hour

    # 统计每一小时内的车流量
    hours = np.arange(0, 24)
    counts = []
    for hour in hours:
        subset = df[(df['start_hour'] <= hour) & (df['stop_hour'] >= hour)]
        counts.append(len(subset))
        
    percs = ["{:.2%}".format(v/sum(counts)) for v in counts]
    return counts, percs

bus_hourflow, bus_hourflow_perc = hour_trafficflow(bus_test_data)
lighttruck_hourflow, lighttruck_hourflow_perc = hour_trafficflow(lighttruck_test_data)
heavytruck_hourflow, heavytruck_hourflow_perc = hour_trafficflow(heavytruck_test_data)

# 合并统计结果
flow_statics = {'hour': np.arange(0, 24), 'bus_flow': bus_hourflow, 'bus_perc': bus_hourflow_perc,
                'lighttruck_flow': lighttruck_hourflow, 'lighttruck_perc': lighttruck_hourflow_perc,
                'heavytruck_flow': heavytruck_hourflow, 'heavytruck_perc': heavytruck_hourflow_perc}
flow_statics = pd.DataFrame(flow_statics)

# 保存结果
# flow_statics.to_excel('2_时间分布特征/' + '24小时流量变化.xlsx', index=False)
flow_statics

,hour,bus_flow,bus_perc,lighttruck_flow,lighttruck_perc,heavytruck_flow,heavytruck_perc
0,0,2264,0.72%,50,1.10%,16,0.64%
1,1,1600,0.51%,58,1.28%,10,0.40%
2,2,1033,0.33%,46,1.01%,11,0.44%
3,3,727,0.23%,49,1.08%,15,0.60%
4,4,633,0.20%,48,1.06%,18,0.72%
5,5,771,0.25%,64,1.41%,40,1.60%
6,6,4218,1.35%,71,1.56%,57,2.28%
7,7,27193,8.71%,129,2.84%,89,3.57%
8,8,28339,9.07%,244,5.37%,119,4.77%
9,9,20035,6.41%,406,8.94%,199,7.97%


2. 24小时路网平均运行速度变化

In [23]:
def hour_meanspeed(df):

    # 统计每一小时内的平均速度
    hours = np.arange(0, 24)
    speeds = []
    for hour in hours:
        subset = df[(df['start_hour'] <= hour) & (df['stop_hour'] >= hour)]
        if subset.empty:
            mean_speed = 0
        else:
            mean_speed = subset['speed_kmh'].mean()
            mean_speed = round(mean_speed, 2)
        speeds.append(mean_speed)

    return speeds

bus_hourspeed = hour_meanspeed(bus_test_data)
lighttruck_hourspeed = hour_meanspeed(lighttruck_test_data)
heavytruck_hourspeed = hour_meanspeed(heavytruck_test_data)

# 合并统计结果
speed_statics = {'hour': np.arange(0, 24), 'bus_speed': bus_hourspeed, 'lighttruck_speed': lighttruck_hourspeed, 'heavytruck_speed': heavytruck_hourspeed}
speed_statics = pd.DataFrame(speed_statics)

print(f"客车平均速度: {round(np.mean(bus_hourspeed), 2)} km/h")
print(f"微轻型货车平均速度: {round(np.mean(lighttruck_hourspeed), 2)} km/h")
print(f"中重型货车平均速度: {round(np.mean(heavytruck_hourspeed), 2)} km/h")

# 保存结果
# speed_statics.to_excel('2_时间分布特征/' + '24小时路网运行速度变化.xlsx', index=False)
speed_statics

客车平均速度: 35.22 km/h
微轻型货车平均速度: 34.17 km/h
中重型货车平均速度: 33.77 km/h


,hour,bus_speed,lighttruck_speed,heavytruck_speed
0,0,37.98,38.79,37.68
1,1,38.01,37.18,38.08
2,2,37.93,37.78,41.61
3,3,37.79,40.75,38.20
4,4,39.79,39.98,38.15
5,5,39.93,40.41,35.47
6,6,36.88,36.35,34.50
7,7,31.90,32.74,28.86
8,8,30.55,30.83,29.87
9,9,33.53,31.31,30.81
